Notebook containing a universal connector to Frontier models - stored in a .py file in the src folder for easy accessibility and imports

# Imports

In [ ]:
import os
import requests
from dotenv import load_dotenv

from openai import OpenAI
import ollama
# from mistralai.client import MistralClient
import google.generativeai as genai
# from anthropic import Anthropic
# !pip install groq
from groq import Groq

# Model Selection Utility

In [78]:
load_dotenv()

True

In [88]:
def create_model_client(model_name):
    """
    Create a client for the specified model based on the provider.
    
    Args:
        model_name (str): Name of the model to use
    
    Returns:
        tuple: (client_object, model_name, provider_name)
    """
    provider_mapping = {
        'openai': ['gpt-', 'text-', 'dall-e'],
        'ollama': ['llama3.2', 'meta-', 'deepseek'],
        'mistral': ['mistral', 'mixtral', 'codestral'],
        'google': ['gemini'],
        'anthropic': ['claude'],
        'groq': ['groq',
        'gemma2-9b-it',
        'llama-3.3-70b-versatile','llama-3.1-8b-instant','llama-guard-3-8b','llama3-70b-8192','llama3-8b-8192', 
        'whisper-large-v3', 'whisper-large-v3-turbo','distil-whisper-large-v3-en']
    }
    
    provider = 'ollama'  # default provider
    for prov, prefixes in provider_mapping.items():
        if any(model_name.startswith(prefix) for prefix in prefixes):
            provider = prov
            break

    # if provider == 'mistral' and 'MISTRAL_API_KEY' not in os.environ:
    #     provider = 'ollama'
    
    if provider == 'openai':
        openai_api_key = os.getenv('OPENAI_API_KEY')
        return OpenAI(api_key=openai_api_key), model_name, 'OpenAI'
    
    elif provider == 'mistral':
        # mistral_api_key = os.getenv('MISTRAL_API_KEY')
        # return MistralClient(api_key=mistral_api_key), model_name, 'Mistral'
        provider == 'ollama'

    if provider == 'ollama':
        models = ollama.list().get('models', [])
        model_exists = any(m.get('name') == model_name for m in models)
        if not model_exists:
            print(f"Model '{model_name}' not found locally. Pulling from Ollama...")
            ollama.pull(model_name)
            print(f"Successfully pulled model: {model_name}")
        return ollama, model_name, 'Ollama'
    
    elif provider == 'google':
        google_api_key = os.getenv('GOOGLE_API_KEY')
        genai.configure(api_key=google_api_key)
        return genai, model_name, 'Google Gemini'
    
    elif provider == 'anthropic':
        anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
        return Anthropic(api_key=anthropic_api_key), model_name, 'Anthropic'
    
    elif provider == 'groq':
        groq_api_key = os.getenv('GROQ_API_KEY')
        return Groq(api_key=groq_api_key), model_name, 'Groq'
    
    else:
        raise ValueError(f"Unsupported model provider: {model_name},{provider}")

In [89]:
def chat_with_model(model_name, user_message, system_message=None):
    """
    Send a message to the specified model and return the response.
    
    Args:
        model_name (str): Name of the model to use
        user_message (str): Message to send to the model
        system_message (str, optional): System instructions/context
        
    Returns:
        str: Model's response
    """
    client, model, provider_name = create_model_client(model_name)
    
    print(f"Using {provider_name} with model: {model}")
        
    messages = []

    if system_message:
        messages.append({"role": "system", "content": system_message})
    
    messages.append({"role": "user", "content": user_message})
    
    provider_handlers = {
        'OpenAI': lambda: client.chat.completions.create(
            model=model,
            messages=messages
        ).choices[0].message.content,
        
        'Ollama': lambda: client.chat(
            model=model, 
            messages=messages
        )['message']['content'],
        
        'Mistral': lambda: client.chat(
            model=model,
            messages=messages
        ).choices[0].message.content,
        
        'Anthropic': lambda: client.messages.create(
            model=model,
            system=system_message if system_message else None,
            messages=[{"role": "user", "content": user_message}]
        ).content[0].text,
        
        'Groq': lambda: client.chat.completions.create(
            model=model,
            messages=messages
        ).choices[0].message.content,

        'Google Gemini': lambda: client.GenerativeModel(model).generate_content(
            [{
                "role": "user" if msg["role"] == "user" else "model", "parts": [msg["content"]]
            } for msg in messages]
        ).text if system_message else client.GenerativeModel(model).generate_content(user_message).text
    }
    
    handler = provider_handlers.get(provider_name)
    
    if handler:
        return handler()
    else:
        return "Unsupported provider"

In [73]:
def get_model_response(model_name, user_message, system_message=None):
    """
    Get a response from a specified model using provided messages.
    
    Args:
        model_name (str): Name of the model to use
        user_message (str): Message to send to the model
        system_message (str, optional): System instructions/context
        
    Returns:
        str: Model's response
    """
    print(f"Using model: {model_name}")
    if system_message:
        print(f"With system prompt: {system_message[:50]}...")
    if user_message:
        print(f"With user prompt: {user_message[:50]}...")
    try:
        response = chat_with_model(model_name, user_message, system_message)
        return response
    except Exception as e:
        print(f"Error: {e}")
        return None

# Testing 

In [90]:
test_user_message="What are the different types of clouds and how to tell them apart?"
test_system_message="You are a helpful AI assistant with expertise in meteorology"

In [ ]:
# get_model_response()
response = get_model_response(
    model_name="gpt-4o", 
    user_message=test_user_message,
    system_message=test_system_message
)

if response:
    print("\nResponse:")
    print(response)

Using model: gpt-4o
With system prompt: You are a helpful AI assistant with expertise in m...
With user prompt: What are the different types of clouds and how to ...


Using OpenAI with model: gpt-4o

Response:
Clouds are classified based on their appearance, altitude, and the process by which they form. Here’s an overview of the main cloud types and how to distinguish them:

1. **High Clouds (Above 20,000 feet or 6,000 meters):**
   - **Cirrus (Ci):** These clouds are thin, wispy, and white, often appearing as streaks across the sky. They usually indicate fair weather, but can also signal an approaching warm front or upper-level moisture.
   - **Cirrostratus (Cs):** These are thin, ice-crystal clouds covering the sky like a veil, often creating a halo around the sun or moon.
   - **Cirrocumulus (Cc):** Small, white patches or ripples with a grainy texture, often resembling fish scales. They rarely cover the entire sky and usually indicate fair, cold weather.

2. **Middle Clouds (6,500 to 20,000 feet or 2,000 to 6,000 meters):**
   - **Altostratus (As):** These clouds form in gray or blue-gray, dense sheets or layers that usually cover most or all of

In [ ]:
response = get_model_response(
    model_name="llama3.2", 
    user_message=test_user_message,
    system_message=test_system_message
)

if response:
    print("\nResponse:")
    print(response)

Using model: llama3.2
With system prompt: You are a helpful AI assistant with expertise in m...
With user prompt: What are the different types of clouds and how to ...
Model 'llama3.2' not found locally. Pulling from Ollama...
Successfully pulled model: llama3.2
Using Ollama with model: llama3.2

Response:
There are many types of clouds, and they can be classified based on their shape, height, and other characteristics. Here's a comprehensive overview of the main cloud types and some tips on how to tell them apart:

**Main Cloud Types:**

1. **Cirrus clouds**: Thin, wispy clouds that appear as feathery streaks across the sky.
	* Height: High altitude (above 20,000 ft)
	* Appearance: White or light gray, thin and wispy
2. **Cumulus clouds**: Puffy, white clouds with flat bases and rounded tops.
	* Height: Low to middle altitude (up to 10,000 ft)
	* Appearance: White, puffy, and cotton-like
3. **Stratus clouds**: Low-level clouds that cover the entire sky like a blanket.
	* Height: Low a

In [82]:
response = get_model_response(
    model_name="gemini-1.5-pro-latest", 
    user_message=test_user_message,
    system_message=test_system_message
)

if response:
    print("\nResponse:")
    print(response)

Using model: gemini-1.5-pro-latest
With system prompt: You are a helpful AI assistant with expertise in m...
With user prompt: What are the different types of clouds and how to ...
Using Google Gemini with model: gemini-1.5-pro-latest

Response:
Clouds are categorized based on their altitude and shape.  Here's a breakdown of the main types and how to distinguish them:

**High-Level Clouds (above 20,000 feet):**

* **Cirrus (Ci):**  These are wispy, feathery clouds made of ice crystals. They often appear as delicate streaks or patches in the sky.  They are often a sign of fair weather, but can sometimes precede a warm front.
* **Cirrocumulus (Cc):**  Small, white, grainy patches or ripples arranged in a regular pattern, often resembling fish scales (hence the nickname "mackerel sky").  They are also made of ice crystals.
* **Cirrostratus (Cs):**  A thin, sheet-like cloud that often covers the entire sky.  It can create a halo effect around the sun or moon due to the refraction of light 

In [91]:
response = get_model_response(
    model_name="llama-3.1-8b-instant", 
    user_message=test_user_message,
    system_message=test_system_message
)

if response:
    print("\nResponse:")
    print(response)

Using model: llama-3.1-8b-instant
With system prompt: You are a helpful AI assistant with expertise in m...
With user prompt: What are the different types of clouds and how to ...
Using Groq with model: llama-3.1-8b-instant

Response:
Cloud classification is based on the shape and height of the clouds. The main types of clouds are classified into two categories: high-level clouds and low-level clouds, with some additional clouds classified as mid-level or vertical clouds.

**High-level clouds (above 20,000 feet)**

1. **Cirrus clouds**: Thin, wispy lines that can appear feathery or cotton-like, often indicating fair weather. They indicate a rising motion of air, usually above 20,000 feet.
2. **Cirrostratus clouds**: Thin, uniform veils that often cover the entire sky and may produce halos around the sun or moon. They can reduce visibility and temperature, usually causing a partial eclipse of the sun.
3. **Cirrocumulus clouds**: Small, rounded masses that are smooth and layered, often a

In [ ]:
# response = get_model_response(
#     model_name="claude", 
#     user_message=test_user_message,
#     system_message=test_system_message
# )

# if response:
#     print("\nResponse:")
#     print(response)

In [ ]:
# response = get_model_response(
#     model_name="mistral", 
#     user_message=test_user_message,
#     system_message=test_system_message
# )

# if response:
#     print("\nResponse:")
#     print(response)